In [0]:
import sys
import importlib
sys.path.append('/Workspace/Users/luisfdiaz@bcp.com.pe/Modulo de Seguimiento/')

import SegScore as rmm
importlib.reload(rmm)

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs,greatest
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

In [0]:
#Mostrar los duplicados
def print_res(sparkf_df):
  import pandas as pd
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  pd_spark_df = sparkf_df.toPandas()
 
  return pd_spark_df

from collections import Counter

def find_repeated_columns(df):
    column_names = df.columns
    column_counts = Counter(column_names)
    repeated_columns = [col for col, count in column_counts.items() if count > 1]
    return repeated_columns

def find_missing_columns(df1, df2):
    columns_df1 = set(df1.columns)
    columns_df2 = set(df2.columns)
    
    missing_in_df2 = columns_df1 - columns_df2
    missing_in_df1 = columns_df2 - columns_df1
    
    print("Columnas en df1 pero no en df2:", missing_in_df2)
    print("Columnas en df2 pero no en df1:", missing_in_df1)

def rezago_codmes_col(base, rezago_mes):
    for i in range(0, rezago_mes + 1, 1):
        base = base.withColumn(f"CODMES_{i}", date_format(add_months(to_date(concat(col("codmes"), lit("01")), "yyyyMMdd"), -i), "yyyyMM"))
        base = base.withColumn(f"CODMES_{i}", col(f"CODMES_{i}").cast("integer"))
    return base

def avance_codmes_col(base, avance_mes):
    for l in range(0, avance_mes + 1, 1):
        base = base.withColumn(f"CODMES_F{l}", date_format(add_months(to_date(concat(col("codmes"), lit("01")), "yyyyMMdd"), l), "yyyyMM"))
        base = base.withColumn(f"CODMES_F{l}", col(f"CODMES_F{l}").cast("integer"))
    return base

In [0]:
hip = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_univ_pre_driver_app_hip')

hip = hip.withColumnRenamed("XB_P3_AJUST", "XB_CAL") \
       .withColumnRenamed("PD_P3_AJUST", "PD_CAL") \
       .withColumnRenamed("SC_P3_AJUST", "SC_CAL")

# Apply the filter conditions
hip_ahlo = hip.filter(
    (col("TIPO_FONDEO") == "FONDEO MVIV") &
    (col("CAMPANIA_AGRUPADA") == "Ahorro Local")
)

## Base Espejo

In [0]:
espejo_total = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_mm_appscore_hip_ahlo')

In [0]:
espejo_total = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_mm_appscore_hip_ahlo')
hip_ahlo.createOrReplaceTempView("hip_ahlo")
espejo_total.createOrReplaceTempView('espejo_total')
hip_ahlo_1 = spark.sql("""
SELECT A.*,
       B.XB_P3,
       B.XB_FINAL_BASE,
       B.FLG_BANCARIZADO AS FLG_BANCARIZADO_CAL,
       B.FLG_DEPEN_TOT2_ST,
       COALESCE(B.SIT_LABORAL_EXP, '99. MISSING') AS SIT_LABORAL_EXP,
       B.SEGMENTO AS SEGMENTO_CAL,
       B.SEGMENTO2_ST,
       B.XB_COMP_NOCLI_CAL,
       B.XB_COMP_NOCLI,
       B.SF3_SF24,
       B.SF3_SF24_A3,
       B.FLAG_CLIENTE_6,
       B.TIPCLASIFRIESGOSBSMAX24,
       B.TIPCLASIFRIESGOSBSMAX24_A3,
       B.UTL_24_V1,
       B.UTL_24_V1_A3,
       B.XB_OPE_NOCLI_CAL,
       B.XB_OPE_NOCLI,
       B.APALANCAMIENTO_VF,
       B.APALANCAMIENTO_VF_A1,
       B.CAT_EDAD_SEX_EST,
       B.CAT_EDAD_SEX_EST_st,
       B.CAT_LAB_ANT,
       B.CAT_LAB_ANT_st,
       B.CTDPLAZOAPROBADO_N,
       B.CTDPLAZOAPROBADO_N_st,
       B.LTV_AJUSTADO,
       B.LTV_AJUSTADO_A3,
       B.ZONA_RIES,
       B.ZONA_RIES_st,
       B.XB_COMP_CLI_CAL,
       B.XB_COMP_CLI,
       B.ATRASOMAX_12,
       B.ATRASOMAX_12_A2,
       B.IGCLETMXCN50F_SUM_12,
       B.IGCLETMXCN50F_SUM_12_J,
       B.MESES_FMSAVMF_24_1000,
       B.MESES_FMSAVMF_24_1000_J,
       B.MESES_PASIVO_ACTIVO_12_1000,
       B.MESES_PASIVO_ACTIVO_12_1000_A3,
       B.RT_IGCLETMXCNFDS_MED_6_12,
       B.RT_IGCLETMXCNFDS_MED_6_12_A1,
       B.RT_PMACTIVO_SUM_6_24,
       B.RT_PMACTIVO_SUM_6_24_A3,
       B.SF12_SF24,
       B.SF12_SF24_D2,
       B.TIPCLASIFRIESGOSBSMAX24_A1,
       B.XB_OPE_CLI_CAL,
       B.XB_OPE_CLI,
       B.APALANCAMIENTO_VF_A3,
       B.FLG_ZONA_RIES,
       B.FLG_ZONA_RIES_A2,
       B.LTV_AJUSTADO_A2,
       B.TIPSITUACIONLABORAL_N,
       B.TIPSITUACIONLABORAL_N_st,
       COALESCE(B.FLAG_CLIENTE_6, 0) AS FLAG_CLIENTE,
       COALESCE(B.FLAG_CLI_PAS_6, 0) AS FLAG_PASIVO,
       COALESCE(B.FLAG_CLI_ACT_6, 0) AS FLAG_ACTIVO,
       B.ING_CASCADA2_2 AS ING_CASCADA_CAL,
       b.ZONA_SEG,
       B.MTOTOTALDEUDA1 AS MTOTOTALDEUDA1_CAL,
       B.PD_P3_AJUST AS PD_ESPEJO,
       B.CODMES_SOLICITUD AS CODMES_SOLICITUD_ESPEJO,
       B.APALANCAMIENTO,
       B.TIPESTCIVIL_N,
       B.TIPESTCIVIL AS TIPESTCIVIL_ESPEJO,
       B.EDAD AS EDAD_ESPEJO,
       B.FLGSEXO,
       B.INGRESO_5TA_TIT,
       B.ANT_LABORAL_MESES,
       B.PD_P1,
       B.XB_P1,
       B.XB_P3_AJUST,
       B.INGRESO_5TA 
FROM hip_ahlo A
LEFT JOIN espejo_total B ON A.CODSOLICITUD = B.CODSOLICITUD
""")

hip_ahlo_1.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = hip_ahlo_1.groupBy("codmes", "codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

## Puntuaciones Tácticas

In [0]:
VER = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_HM_ADM_HIP_AHLO_Q424_202410')

In [0]:
puntuacion_reciente = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_hm_adm_hip_ahlo_score')

hip_ahlo_1.createOrReplaceTempView("hip_ahlo_1")
puntuacion_reciente.createOrReplaceTempView('puntuacion_reciente')
hip_ahol_2 = spark.sql("""
SELECT A.*, 
       B.PD_2024Q4 AS PD_2024Q4,
       -LOG((1/B.PD_2024Q4)-1) AS SC_2024Q4,
       C.PD_2024Q4 AS PD_2024Q4v2,
       -LOG((1/C.PD_2024Q4)-1) AS SC_2024Q4v2,
       case when A.CODMES<=202307 THEN B.PD_2024Q4 ELSE  C.PD_2024Q4 END as PD_2024Q4vf,
       case when A.CODMES<=202307 THEN B.EXP_PCT_EVOL_SHIP_U3M_RT_U6M ELSE  C.EXP_PCT_EVOL_SHIP_U3M_RT_U6M END as EXP_PCT_EVOL_SHIP_U3M_RT_U6M_FAB,
       case when A.CODMES<=202307 THEN B.EXP_PCT_EVOL_SHIP_U3M_RT_U6M_J ELSE  C.EXP_PCT_EVOL_SHIP_U3M_RT_U6M_J END as EXP_PCT_EVOL_SHIP_U3M_RT_U6M_J_FAB,
       case when A.CODMES<=202307 THEN B.pd_disef ELSE  C.pd_disef END as pd_disef_FAB,
       case when A.CODMES<=202307 THEN B.PD_DISEF_D2 ELSE  C.PD_DISEF_D2 END as PD_DISEF_D2_FAB,
       case when A.CODMES<=202307 THEN B.XB_IDIO_2024Q4 ELSE  C.XB_IDIO_2024Q4 END as XB_IDIO_2024Q4_FAB,
       case when A.CODMES<=202307 THEN B.XB_FWL_2024Q4 ELSE  C.XB_FWL_2024Q4 END as XB_FWL_2024Q4_FAB,
       C.XB_CEF_FWL AS XB_CEF_FWL_FAB
FROM hip_ahlo_1 A
LEFT JOIN catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_HM_ADM_HIP_AHLO_Q424_202410 B
ON A.CODCLAVEOPECTA = B.CODCLAVEOPECTA AND A.CODMES = B.CODMES
LEFT JOIN puntuacion_reciente C
ON A.CODCLAVEOPECTA = C.CODCLAVEOPECTA AND A.CODMES = C.CODMES
""")

hip_ahol_2.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = hip_ahol_2.groupBy("codmes", "codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
rbm_nft = spark.table('catalog_lhcl_prod_bcp.bcp_ddv_rbmrbmper_solicitudch_vu.hm_solicitudcreditohipotecariorbm')

rbm_max = rbm_nft.groupBy('codsolicitudcredito').agg(F.max('CODMES').alias("codmes"))

rbm_ltv_nueva = rbm_max.join(
    rbm_nft.selectExpr("codmes", "codsolicitudcredito", "pctfinanciamientocredito as NFT_LTV_DEF"),
    on=['codsolicitudcredito', 'codmes'],
    how='left'
)

hip_ahol_2 = hip_ahol_2.join(
    rbm_ltv_nueva.selectExpr("codsolicitudcredito as codsolicitud", "NFT_LTV_DEF"),
    on='codsolicitud',
    how='left'
)

hip_ahol_2.persist(StorageLevel.MEMORY_AND_DISK).count()

dup = hip_ahol_2.groupBy("codmes", "codclaveopecta").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
PD_SEG='PD_2024Q4'
SCORE='SC_2024Q4'
PAUTA=386

hip_ahol_2 = hip_ahol_2.withColumn("ING_CASCADA_R", when(col("ING_CASCADA_CAL").isNull() | (col("ING_CASCADA_CAL") == lit('.')), '99. Missing')
                   .when(col("ING_CASCADA_CAL") <= 1500, '1. [0 - 1,500]')
                   .when(col("ING_CASCADA_CAL") <= 2000, '2. <1,500 - 2,000]')
                   .when(col("ING_CASCADA_CAL") <= 2500, '3. <2,000 - 2,500]')
                   .when(col("ING_CASCADA_CAL") <= 4000, '4. <2,500 - 4,000]')
                   .otherwise('5. <4,000 - ++]'))

# Add the ANT_LABORAL column
hip_ahol_2 = hip_ahol_2.withColumn("ANT_LABORAL", when(col("ANTIGUEDAD_LABORAL_T").isNull(), '99. Missing')
                   .when(col("ANTIGUEDAD_LABORAL_T") <= 24, '1. <0-24]')
                   .when(col("ANTIGUEDAD_LABORAL_T") <= 84, '2. <24-84]')
                   .otherwise('3. >84'))

# Add the END_COUTA_INGCASCADA column
hip_ahol_2 = hip_ahol_2.withColumn("END_COUTA_INGCASCADA", coalesce(col("MTOTOTALDEUDA1_CAL"), lit(0)) / coalesce(col("ING_CASCADA_CAL"), lit(0)))

# Add the END_COUTA_INGCASCADA_R column
hip_ahol_2 = hip_ahol_2.withColumn("END_COUTA_INGCASCADA_R", when(col("ING_CASCADA_CAL").isNull(), '99. Missing')
                   .when(col("MTOTOTALDEUDA1_CAL").isNull(), '99. Missing')
                   .when(col("END_COUTA_INGCASCADA") <= 0, '97. <=0%')
                   .when(col("END_COUTA_INGCASCADA") <= 0.20, '1. <=20%')
                   .when(col("END_COUTA_INGCASCADA") <= 0.30, '2. <=30%')
                   .when(col("END_COUTA_INGCASCADA") <= 0.40, '3. <=40%')
                   .when(col("END_COUTA_INGCASCADA") <= 1, '4. <=100%')
                   .otherwise('98. >100%'))

# Add the RSC column
hip_ahol_2 = hip_ahol_2.withColumn("RSC", when(col(SCORE).isNull(), '99. Missing')
                   .when(col(SCORE) <= 335, '01. <0 - 335]')
                   .when(col(SCORE) <= 385, '02. <335 - 385]')
                   .when(col(SCORE) <= 435, '03. <385 - 435]')
                   .when(col(SCORE) <= 495, '04. <435 - 495]')
                   .when(col(SCORE) > 495, '05. <495 - ++]')
                   .otherwise('98. Otro'))

# Add the RPD column
hip_ahol_2 = hip_ahol_2.withColumn("RPD", when(col(PD_SEG).isNull(), '99. Missing')
                   .when(col(PD_SEG) <= 0.0038411, '01. [0 - 0.38%]')
                   .when(col(PD_SEG) <= 0.0107884, '02. <0.38 - 1.08%]')
                   .when(col(PD_SEG) <= 0.0252832, '03. <1.08 - 2.53%]')
                   .when(col(PD_SEG) <= 0.0581086, '04. <2.53 - 5.81%]')
                   .when(col(PD_SEG) > 0.0581086, '05. <5.81 - 100%]')
                   .otherwise('98. Otro'))

# Add the RANGO_MONTO column
hip_ahol_2 = hip_ahol_2.withColumn("RANGO_MONTO", when(col("MTOAPROBADO_SOLES").isNull(), '99. Missing')
                   .when(col("MTOAPROBADO_SOLES") <= 100000, '1. [0 - 100K]')
                   .when(col("MTOAPROBADO_SOLES") <= 150000, '2. <100 - 150K]')
                   .when(col("MTOAPROBADO_SOLES") <= 200000, '3. <150 - 200K]')
                   .when(col("MTOAPROBADO_SOLES") <= 250000, '4. <200 - 250K]')
                   .otherwise('5. <250K - ++]'))

# Add the PAUTA column
hip_ahol_2 = hip_ahol_2.withColumn("PAUTA", when(col("SC_CAL") >= PAUTA, 1).otherwise(0))

# Add the SIT_LABORAL_EXP_F column
hip_ahol_2 = hip_ahol_2.withColumn("SIT_LABORAL_EXP_F", when(col("FLG_PDH_2_3") == 1, 'DEPENDIENTE').otherwise(col("SIT_LABORAL_EXP")))

# Add the MISS_PD column
hip_ahol_2 = hip_ahol_2.withColumn("MISS_PD", when(col(PD_SEG).isNull(), 1).otherwise(0))

# Add the MISS_MONTO column
hip_ahol_2 = hip_ahol_2.withColumn("MISS_MONTO", when(col("MTOAPROBADO_SOLES").isNull(), 1).otherwise(0))

hip_ahol_2 = hip_ahol_2.withColumn(
    "PD_RBM", 
    when(col("codmes") <= 202307, col("PD_ESTADOS")).otherwise(col("CSOL_PD_RBM"))
)

hip_ahol_2 = hip_ahol_2.withColumn("CAT_EDAD_SEX_EST_Q", when(col("CAT_EDAD_SEX_EST") == "CR1", "1 Soltero - Menor a 26 años")
                   .when(col("CAT_EDAD_SEX_EST") == "CR2", "2 Soltero - 26 a 42 años - Hombre")
                   .when(col("CAT_EDAD_SEX_EST") == "CR3", "5 Soltero - 26 a 42 años - Mujer")
                   .when(col("CAT_EDAD_SEX_EST") == "CR4", "4 Soltero - Mayor a 42 años")
                   .when(col("CAT_EDAD_SEX_EST") == "CR5", "3 Casado o Otros - Menor a 39 años")
                   .when(col("CAT_EDAD_SEX_EST") == "CR6", "6 Casado o Otros - 39 a más años")
                   .when(col("CAT_EDAD_SEX_EST") == "NCR1", "14 Casado o Otros - Mujer")
                   .when(col("CAT_EDAD_SEX_EST") == "NCR2", "12 Casado o Otros - Hombre")
                   .when(col("CAT_EDAD_SEX_EST") == "NCR3", "8 Soltero - Menor a 26 años - Mujer")
                   .when(col("CAT_EDAD_SEX_EST") == "NCR4", "10 Soltero - 26 a 33 años - Mujer")
                   .when(col("CAT_EDAD_SEX_EST") == "NCR5", "13 Soltero - 34 a más años - Mujer")
                   .when(col("CAT_EDAD_SEX_EST") == "NCR6", "7 Soltero - Menor a 26 años - Hombre")
                   .when(col("CAT_EDAD_SEX_EST") == "NCR7", "9 Soltero - 26 a 33 años - Hombre")
                   .when(col("CAT_EDAD_SEX_EST") == "NCR8", "11 Soltero - 34 a más años - Hombre")
                   .otherwise("Missing"))

hip_ahol_2 = hip_ahol_2.withColumn("flag_perfil", when(col("MTOCEMCLISOL_6")/col("MTOAPROBADO_SOLES") <= 0.000430813, 1).otherwise(0))
hip_ahol_2 = hip_ahol_2.withColumn("FLG_VC_VIVIENDA_RBM", F.when(F.col("PRS_ValorComVivienda") <= 355100, 1).otherwise(0))

hip_ahol_2 = hip_ahol_2.withColumn(
    "RANGO_MTOVIVI_F",
    when(col("PRS_ValorComVivienda").isNull(), "99. Missing")
    .when(col("PRS_ValorComVivienda") <= 67400, "1. [0 - 67K]")
    .when(col("PRS_ValorComVivienda") <= 96200, "2. <67 - 96K]")
    .when(col("PRS_ValorComVivienda") <= 144000, "3. <96 - 144K]")
    .when(col("PRS_ValorComVivienda") <= 239800, "4. <144 - 239K]")
    .when(col("PRS_ValorComVivienda") <= 355100, "5. <239 - 355K]")
    .when(col("PRS_ValorComVivienda") <= 479300, "6. <355 - 479K]")
    .otherwise("7. <479K - ++]")
)

hip_ahol_2 = hip_ahol_2.withColumn("PAUTA_RBM", F.when(F.col("SC_CAL") >= 386, 1).otherwise(0))

#Score final: 
from pyspark.sql.functions import col, log, exp, round
hip_ahol_2 = hip_ahol_2.withColumn("XB_2024Q4vf", -log((1 / col("PD_2024Q4vf")) - 1))
hip_ahol_2 = hip_ahol_2.withColumn("SC_2024Q4vf", round(174.25 - 57.708 * col("XB_2024Q4vf"), 1))
hip_ahol_2 = hip_ahol_2.withColumn("PAUTA_2024Q4vf", F.when(F.col("SC_2024Q4vf") >= 386, 1).otherwise(0))

hip_ahol_2 = hip_ahol_2.withColumn("CEM6_MTO_T", F.col("MTOCEMCLISOL_6") / F.col("MTOAPROBADO_SOLES") * 1.000)

# Crear el campo RNG_CEM6_MTO con las condiciones especificadas
hip_ahol_2 = hip_ahol_2.withColumn("RNG_CEM6_MTO", 
    F.when(F.col("CEM6_MTO_T").isNull(), "99. Missing")
    .when(F.col("CEM6_MTO_T") <= 0.000430813, "01. [0 - 0.04%]")
    .when(F.col("CEM6_MTO_T") <= 0.0018687401, "02. <0.04- 0.18%]")
    .when(F.col("CEM6_MTO_T") <= 0.0041421573, "03. <0.18 - 0.4%]")
    .otherwise("04. <0.4 a mas]"))

ltv_malita = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T45988_EXT_SEG_AGRUP_LTV_202504')

hip_ahol_2 = hip_ahol_2.alias("A") \
    .join(ltv_malita.alias("F"), F.col("A.CODSOLICITUD") == F.col("F.CODSOLICITUD"), "left") \
    .join(hip_ahol_2.alias("G"), (F.col("A.CODSOLICITUD") == F.col("G.CODSOLICITUD")) & (F.col("A.CODCLAVEOPECTA") == F.col("G.CODCLAVEOPECTA")), "left") \
    .select("A.*", F.coalesce(F.col("F.LTV"), F.col("G.LTV_RBM")).alias("LTV_CORREGIDO_0") ,F.col("F.LTV").alias("LTV_EXT"))

# hip_ahol_2 = hip_ahol_2.withColumn("NFT_LTV", col("NFT_LTV").cast("double"))
hip_ahol_2 = hip_ahol_2.withColumn("LTV_CORREGIDO_VF", when(col("CODMES") >= 202308,  col("NFT_LTV")).otherwise(col("LTV_CORREGIDO_0")))

# Rango de LTV
hip_ahol_2 = hip_ahol_2.withColumn(
    "LTV_CORREGIDO_0_R",
    F.when(F.col("LTV_CORREGIDO_0").isNull(), '99. Missing')
     .when(F.col("LTV_CORREGIDO_0") > 1, '7. <100 - +]')
     .when(F.col("LTV_CORREGIDO_0") > 0.90, '6. <90 - 100%]')
     .when(F.col("LTV_CORREGIDO_0") > 0.80, '5. <80 - 90%]')
     .when(F.col("LTV_CORREGIDO_0") > 0.70, '4. <70 - 80%]')
     .when(F.col("LTV_CORREGIDO_0") > 0.60, '3. <60 - 70%]')
     .when(F.col("LTV_CORREGIDO_0") > 0.50, '2. <50 - 60%]')
     .otherwise('1. [0 - 50%]')
)

# Crear el campo SEGMENTO_CAL_2024Q4 con las condiciones especificadas
hip_ahol_2 = hip_ahol_2.withColumn("SEGMENTO_CAL_2024Q4", 
    F.when((F.col("SIT_LABORAL_EXP_F") == "DEPENDIENTE") & 
           (~F.col("LTV_CORREGIDO_0_R").isin("7. <100 - +]", "6. <90 - 100%]", "5. <80 - 90%]")), "1. Dep+LTV<80%")
    .when((F.col("SIT_LABORAL_EXP_F") == "DEPENDIENTE") & 
          (F.col("LTV_CORREGIDO_0_R").isin("7. <100 - +]", "6. <90 - 100%]", "5. <80 - 90%]")), "2. Dep+LTV>80%")
    .when((F.col("SIT_LABORAL_EXP_F") != "DEPENDIENTE") & 
          (~F.col("LTV_CORREGIDO_0_R").isin("7. <100 - +]", "6. <90 - 100%]", "5. <80 - 90%]")), "3. No Dep+LTV<80%")
    .when((F.col("SIT_LABORAL_EXP_F") != "DEPENDIENTE") & 
          (F.col("LTV_CORREGIDO_0_R").isin("7. <100 - +]", "6. <90 - 100%]", "5. <80 - 90%]")), "4. No Dep+LTV>80%"))

hip_ahol_2 = hip_ahol_2.withColumn(
    "SEGMENTO_CAL_2024Q4_v2", 
    when((col("SIT_LABORAL_EXP_F") == "DEPENDIENTE") & (col("LTV_CORREGIDO_VF") <= 0.80), "1. Dep+LTV<80%")
    .when((col("SIT_LABORAL_EXP_F") == "DEPENDIENTE") & (col("LTV_CORREGIDO_VF") > 0.80), "2. Dep+LTV>80%")
    .when((col("SIT_LABORAL_EXP_F") != "DEPENDIENTE") & (col("LTV_CORREGIDO_VF") <= 0.80), "3. No Dep+LTV<80%")
    .when((col("SIT_LABORAL_EXP_F") != "DEPENDIENTE") & (col("LTV_CORREGIDO_VF") > 0.80), "4. No Dep+LTV>80%")
)

hip_ahol_2 = hip_ahol_2.withColumn("FLG_SEGMENTO_CAL_1", F.when(F.col("SEGMENTO_CAL_2024Q4_v2") == "1. Dep+LTV<80%", 1).otherwise(0))
hip_ahol_2 = hip_ahol_2.withColumn("FLG_SEGMENTO_CAL_2", F.when(F.col("SEGMENTO_CAL_2024Q4_v2") == "2. Dep+LTV>80%", 1).otherwise(0))
hip_ahol_2 = hip_ahol_2.withColumn("FLG_SEGMENTO_CAL_3", F.when(F.col("SEGMENTO_CAL_2024Q4_v2") == "3. No Dep+LTV<80%", 1).otherwise(0))
hip_ahol_2 = hip_ahol_2.withColumn("FLG_SEGMENTO_CAL_4", F.when(F.col("SEGMENTO_CAL_2024Q4_v2") == "4. No Dep+LTV>80%", 1).otherwise(0))

hip_ahol_2 = hip_ahol_2.withColumn("Tipo_excepcion_F", 
                   when(col("flg_PlanAhorro").isNull(), "sin excepcion")
                   .when(col("flg_PlanAhorro") == 1, "Plan de Ahorro")
                   .otherwise("Resto de excepciones"))

hip_ahol_2 = hip_ahol_2.withColumn('PD_FINAL_BASE', expr("1 / (1 + exp(-XB_FINAL_BASE))"))

In [0]:
hip_ahol_2.createOrReplaceTempView("hip_ahol_2")
spark.sql('''
    CREATE OR REPLACE TABLE
        catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_hm_app_hipahlo
    USING DELTA LOCATION 
        'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/T39290/Seguimiento/02_Applicant_Hipotecario/hm_app_hipahlo'
    AS SELECT * FROM hip_ahol_2
    '''
)